# Deploy and Test your Model in 7 Easy Steps
## Step 1: Get Access to your Registered Model in your Workspace


In [14]:
# Check core SDK version number
import azureml.core

print("SDK version:", azureml.core.VERSION)

SDK version: 1.7.0


In [15]:

from azureml.core import Workspace

ws = Workspace.from_config()
print(ws.name, ws.resource_group, ws.location, ws.subscription_id, sep='\n')

mlserviceworkspace
mlserviceworkspace
westus2
601f4351-33bb-4d76-96ca-886940409b3d


In [4]:
# The below code lists all the models in your ML workspace
for model in ws.models:
    print(model)

sklearn_regression_model.pkl
liveliness_detection1
automlux
tf-dnn-mnist
automl-local-regression-wipro
amlstudio-experiment-created-o
battery_failure_predictor
face_liveliness
AutoMLef8e3168e9
mask_rcnn_horovod
entity-extraction-from-text
toystfod
keras_animals
keras_pets_1
mnist
demo-model
textDNN-20News
azure-service-classifier
AutoML6129f25231
german-credit
AdultCensus.mml
best_model_local
tf-dnn-mnist-phonepe
amlstudio-automobile-price-dat
liveliness_detection
AutoMLca4f003cf9
liveliness_model
AutoML3ad4cb6ac9
AutoMLf11867afa0
fraud_detection
entityextraction
fruitspredictor
keras_
keras_pets_2
tfod
keras_pets
azure-service-classifier-rl
inception
keras-mlp-mnist
german-credit-scb


In [16]:
from azureml.core import Model
model = Model(ws,"german-credit-model")
model

Model(workspace=Workspace.create(name='mlserviceworkspace', subscription_id='601f4351-33bb-4d76-96ca-886940409b3d', resource_group='mlserviceworkspace'), name=german-credit-model, id=german-credit-model:2, version=2, tags={}, properties={})

## Step 2 - Create an Inference Environment

In [17]:
from azureml.core.conda_dependencies import CondaDependencies
from azureml.core.environment import Environment

myenv = Environment("GermanCreditInference")

conda_dep = CondaDependencies()
conda_dep.add_conda_package("scikit-learn")
conda_dep.add_conda_package("joblib")
conda_dep.add_pip_package("azureml-defaults")
conda_dep.add_pip_package("azureml-core")
conda_dep.add_pip_package("inference-schema")

# Adds dependencies to PythonSection of myenv
myenv.python.conda_dependencies=conda_dep

## Step 3 - Write the Scoring Script

1.Init method loads the model in a global object

2.Run method scores the incoming data against the model and returns results.

In [5]:
with open('./score.py', 'r') as f:
    print(f.read())

import json
import os
import numpy as np
import pandas as pd
import joblib
from inference_schema.schema_decorators import input_schema, output_schema
from inference_schema.parameter_types.standard_py_parameter_type import StandardPythonParameterType

def init():
    global model
    
    # Update to your model's filename
    model_filename = "model.pkl"

    # AZUREML_MODEL_DIR is injected by AML
    model_dir = os.getenv('AZUREML_MODEL_DIR')

    print("Model dir:", model_dir)
    print("Model filename:", model_filename)
    
    model_path = os.path.join(model_dir, model_filename)

    # Replace this line with your model loading code
    model = joblib.load(model_path)

# Define some sample data for automatic generation of swagger interface
input_sample = [{
    "Age": 20,
    "Sex": "male",
    "Job": 0,
    "Housing": "own",
    "Saving accounts": "little",
    "Checking account": "little",
    "Credit amount": 100,
    "Duration": 48,
    "Purpose": "radio/TV"
  }]
output_sample =

## Step 4: Create an Inference Configuration

An Inference Configuration is your Scoring script + Inference environment

In [18]:
from azureml.core.model import InferenceConfig

inference_config = InferenceConfig(entry_script="score.py",
                                   environment=myenv)

## Step 5 - Provision the AKS Compute Target

In [9]:
from azureml.core.compute import AksCompute, ComputeTarget
from azureml.core.webservice import Webservice, AksWebservice
# Use the default configuration (can also provide parameters to customize)
prov_config = AksCompute.provisioning_configuration()
aks_config = AksWebservice.deploy_configuration()

aks_name = 'akscompute' 
# Create the cluster
aks_target = ComputeTarget.create(workspace = ws, 
                                  name = aks_name, 
                                  provisioning_configuration = prov_config)

In [14]:
%%time
aks_target.wait_for_completion(show_output = True)
print(aks_target.provisioning_state)
print(aks_target.provisioning_errors)

Creating......................................................................................
SucceededProvisioning operation finished, operation "Succeeded"
Succeeded
None
CPU times: user 566 ms, sys: 64.8 ms, total: 630 ms
Wall time: 7min 24s


In [19]:
from azureml.core.webservice import Webservice, AksWebservice
from azureml.core.compute import AksCompute, ComputeTarget
aks_name = 'akscompute' 
aks_target = ComputeTarget(workspace=ws, name=aks_name)
aks_target

AksCompute(workspace=Workspace.create(name='mlserviceworkspace', subscription_id='601f4351-33bb-4d76-96ca-886940409b3d', resource_group='mlserviceworkspace'), name=akscompute, id=/subscriptions/601f4351-33bb-4d76-96ca-886940409b3d/resourceGroups/mlserviceworkspace/providers/Microsoft.MachineLearningServices/workspaces/mlserviceworkspace/computes/akscompute, type=AKS, provisioning_state=Succeeded, location=westus2, tags=None)

## Step 6 - Deploy the Model on AKS Cluster

In [10]:
%%time
aks_service_name ='mlops-german-credit'
aks_config = AksWebservice.deploy_configuration()
aks_service = Model.deploy(workspace=ws,
                           name=aks_service_name,
                           models=[model],
                           inference_config=inference_config,
                           deployment_config=aks_config,
                           deployment_target=aks_target,
                           overwrite=True)

aks_service.wait_for_deployment(show_output = True)
print(aks_service.state)

Running.......................
Succeeded
AKS service creation operation finished, operation "Succeeded"
Healthy
CPU times: user 397 ms, sys: 57.9 ms, total: 455 ms
Wall time: 2min 22s


In [20]:
aks_service = AksWebservice(ws, name = aks_service_name)

In [21]:
aks_service.scoring_uri

'http://20.191.115.183:80/api/v1/service/mlops-german-credit/score'

In [22]:
aks_service.get_keys()

('ysYsWIRE4NXdOwTAVNwcKvP0JbQAWp4Y', 'WLgrXbPCmE9IxrR0Fh8t7YZwsA4dBxE6')

## Step 7 - Test your Model in AKS

Use the aks-endpoint.http(tests folder) file or Postman or  REST call to test the scoring endpoint